In [1]:
import h5py

In [2]:
with h5py.File('Datasets/MNIST/train.hdf5') as f:
    X_train = f['X'][:]
    y_train = f['y'][:]
with h5py.File('Datasets/MNIST/test.hdf5') as f:
    X_test = f['X'][:]
    y_test = f['y'][:]

In [3]:
X_train.shape

(55000, 784)

In [4]:
y_train.shape

(55000, 10)

In [5]:
X_test.shape

(10000, 784)

In [6]:
y_test.shape

(10000, 10)

In [7]:
import numpy as np

In [8]:
def get_next_batch(X, y, size):
    m = X.shape[0]
    index = np.random.permutation(np.arange(m))
    for i in range(m//size):
        batch_i = index[i*size: i*size+size]
        yield X[batch_i], y[batch_i]

In [9]:
import tensorflow as tf

In [10]:
Learning_Rate = 0.001

In [28]:
tf.reset_default_graph()

In [29]:
def conv_layer(input, size_in, size_out):
    initializer = tf.contrib.layers.xavier_initializer()
    w = tf.Variable(initializer([5, 5, size_in, size_out]))
    b = tf.Variable(tf.constant(0.1, shape=[size_out]))
    
    conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
    act = tf.nn.relu(conv + b)
    
    return act

In [30]:
def fc_layer(input, size_in, size_out):
    initializer = tf.contrib.layers.xavier_initializer()
    w = tf.Variable(initializer([size_in, size_out]))
    b = tf.Variable(tf.constant(0.1, shape=[size_out]))
    
    act = tf.matmul(input, w) + b

    return act

In [31]:
X = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

In [32]:
X_reshape = tf.reshape(X, shape=[-1, 28, 28, 1])

In [33]:
conv1 = conv_layer(X_reshape, 1, 32)
pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

conv2 = conv_layer(pool1, 32, 64)
conv_out = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

In [34]:
flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])

fc1 = fc_layer(flattened, 7 * 7 * 64, 256)
relu = tf.nn.relu(fc1)
logits = fc_layer(relu, 256, 10)

In [35]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))

train_step = tf.train.AdamOptimizer(Learning_Rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [36]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1, 101):
        for batch in get_next_batch(X_train, y_train, 64):
            _ = sess.run(train_step, {X:batch[0], y:batch[1]})
        if epoch % 10 == 0:
            test_acc, test_loss = sess.run([accuracy, loss], {X:X_test, y:y_test})
            
            batch = get_next_batch(X_train, y_train, 1000).__next__() #get 1000 random data from training(single batch of 1000)
            train_acc, train_loss = sess.run([accuracy, loss], {X:batch[0], y:batch[1]})
            
            print("Epoch", epoch, "Train Acc:", np.round(train_acc, 5), \
                  "Train Loss:", np.round(train_loss, 5), \
                  "Test Acc:", np.round(test_acc, 5), "Test Loss:", np.round(test_loss, 5))

Epoch 10 Train Acc: 0.998 Train Loss: 0.00344 Test Acc: 0.9907 Test Loss: 0.03719
Epoch 20 Train Acc: 1.0 Train Loss: 0.00023 Test Acc: 0.9918 Test Loss: 0.04087
Epoch 30 Train Acc: 0.999 Train Loss: 0.00456 Test Acc: 0.9903 Test Loss: 0.05969
Epoch 40 Train Acc: 1.0 Train Loss: 0.0 Test Acc: 0.9936 Test Loss: 0.04873
Epoch 50 Train Acc: 1.0 Train Loss: 0.0 Test Acc: 0.9929 Test Loss: 0.06447
Epoch 60 Train Acc: 1.0 Train Loss: 0.0 Test Acc: 0.9929 Test Loss: 0.07148
Epoch 70 Train Acc: 1.0 Train Loss: 0.0 Test Acc: 0.9933 Test Loss: 0.07868
Epoch 80 Train Acc: 0.998 Train Loss: 0.00406 Test Acc: 0.9917 Test Loss: 0.06956
Epoch 90 Train Acc: 1.0 Train Loss: 0.00033 Test Acc: 0.9927 Test Loss: 0.08106
Epoch 100 Train Acc: 1.0 Train Loss: 3e-05 Test Acc: 0.9918 Test Loss: 0.07418
